In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!wget --no-check-certificate https://www.dropbox.com/s/37u83g55p19kvrl/clean-phone-data-for-students.csv

--2024-10-18 17:20:14--  https://www.dropbox.com/s/37u83g55p19kvrl/clean-phone-data-for-students.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.85.18, 2620:100:6035:18::a27d:5512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.85.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/8h8hvsw9uj6o0524lfe4i/clean-phone-data-for-students.csv?rlkey=lwv5xbf16jerehnv3lfgq5ue6 [following]
--2024-10-18 17:20:15--  https://www.dropbox.com/scl/fi/8h8hvsw9uj6o0524lfe4i/clean-phone-data-for-students.csv?rlkey=lwv5xbf16jerehnv3lfgq5ue6
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucdedf4e0eb6d25d70cf7d3ba86a.dl.dropboxusercontent.com/cd/0/inline/CcvYCdrpUzf4rZ31Bb6Pq3md1AQHGohvY0vHqfZr22RVdv4Enqaajp4_e2DX4fDmCAzNz3UuH4OXTQiU4NiNOCI9FDO_TCWo07KQqITZCRyrb6R1Zw59GWRtmRAQBLoUOWY/file# [following]
--2024-10-18 17:20:16--  https://ucdedf4e0eb6d25d70cf7d3

# Import Libs

In [3]:
%matplotlib inline
import pandas as pd
import sklearn
import numpy as np
from IPython.display import display

import matplotlib.pyplot as plt

# Loading data

In [4]:
data_df = pd.read_csv('clean-phone-data-for-students.csv')
# Show the top 5 rows
display(data_df.head())
# Summarize the data
data_df.describe()

,Sentence Utterance,Action,Object
0,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,enquire,payment
1,internet ยังความเร็วอยุ่เท่าไหร ครับ,enquire,package
2,ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้...,report,suspend
3,พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อ...,enquire,internet
4,ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโ...,report,phone_issues


,Sentence Utterance,Action,Object
count,16175,16175,16175
unique,13389,10,33
top,บริการอื่นๆ,enquire,service
freq,97,10377,2525


In [5]:
# เปลี่ยนชื่อคอลัมน์จาก " Sentence Utterance" เป็น "Sentence_Utterance"
data_df.rename(columns={"Sentence Utterance": "Sentence_Utterance"}, inplace=True)

# ตรวจสอบชื่อคอลัมน์เพื่อยืนยันการเปลี่ยนแปลง
print(data_df.columns)

Index(['Sentence_Utterance', 'Action', 'Object'], dtype='object')


### Data Cleaning

# TODO 1

In [6]:
### To Do 1 ###
display(data_df.describe())
display(data_df.Object.unique())
display(data_df.Action.unique())

,Sentence_Utterance,Action,Object
count,16175,16175,16175
unique,13389,10,33
top,บริการอื่นๆ,enquire,service
freq,97,10377,2525


array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nonTrueMove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd',
       'TrueMoney', 'garbage', 'Payment', 'IDD', 'ringtone', 'Idd',
       'rate', 'loyalty_card', 'contact', 'officer', 'Balance', 'Service',
       'Loyalty_card'], dtype=object)

array(['enquire', 'report', 'cancel', 'Enquire', 'buy', 'activate',
       'request', 'Report', 'garbage', 'change'], dtype=object)

In [7]:
import pandas as pd

# แปลงคอลัมน์ Action และ Object เป็นตัวพิมพ์เล็กเพื่อลบ duplications
data_df['Object'] = data_df['Object'].str.lower()
data_df['Action'] = data_df['Action'].str.lower()

# ลบ whitespace ที่ไม่ต้องการจากคอลัมน์ Sentence_Utterance
data_df['Sentence_Utterance'] = data_df['Sentence_Utterance'].str.strip()
data_df['Sentence_Utterance'] = data_df['Sentence_Utterance'].str.replace(r'\s+', ' ', regex=True)

# ลบประโยคที่ซ้ำกันในคอลัมน์ Sentence_Utterance
data_df = data_df.drop_duplicates(subset='Sentence_Utterance')

# แปลง Action และ Object เป็น Label Encoding
# สร้าง dictionary สำหรับ Label Encoding
action_labels = {label: idx for idx, label in enumerate(data_df['Action'].unique())}
object_labels = {label: idx for idx, label in enumerate(data_df['Object'].unique())}

# สร้าง Label Encoded columns
data_df['Action_Label'] = data_df['Action'].map(action_labels)
data_df['Object_Label'] = data_df['Object'].map(object_labels)

# ดูผลลัพธ์หลังการทำความสะอาดข้อมูลและการแปลง labels
print("Label Encodings for Actions:", action_labels)
print("Label Encodings for Objects:", object_labels)
print(data_df.head())

# ตรวจสอบจำนวนข้อมูลและ unique labels หลังการทำความสะอาด
print("Total rows after cleaning:", len(data_df))
print("Unique Actions:", data_df['Action'].nunique())
print("Unique Objects:", data_df['Object'].nunique())

# ดูสรุปของ DataFrame
print(data_df.describe())

# ดูค่าที่ได้ในแต่ละคอลัมน์
print("Action Label Mapping:", action_labels)
print("Object Label Mapping:", object_labels)

Label Encodings for Actions: {'enquire': 0, 'report': 1, 'cancel': 2, 'buy': 3, 'activate': 4, 'request': 5, 'garbage': 6, 'change': 7}
Label Encodings for Objects: {'payment': 0, 'package': 1, 'suspend': 2, 'internet': 3, 'phone_issues': 4, 'service': 5, 'nontruemove': 6, 'balance': 7, 'detail': 8, 'bill': 9, 'credit': 10, 'promotion': 11, 'mobile_setting': 12, 'iservice': 13, 'roaming': 14, 'truemoney': 15, 'information': 16, 'lost_stolen': 17, 'balance_minutes': 18, 'idd': 19, 'garbage': 20, 'ringtone': 21, 'rate': 22, 'loyalty_card': 23, 'contact': 24, 'officer': 25}
                                  Sentence_Utterance   Action        Object  \
0  <PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counter...  enquire       payment   
1               internet ยังความเร็วอยุ่เท่าไหร ครับ  enquire       package   
2  ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้ ค่ะ   report       suspend   
3  พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อง...  enquire      internet   
4  ฮาโหล คะ พอดีว่าเมื่อวาน

In [8]:
display(data_df[['Sentence_Utterance', 'Action', 'Object', 'Action_Label', 'Object_Label']])

,Sentence_Utterance,Action,Object,Action_Label,Object_Label
0,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counter...,enquire,payment,0,0
1,internet ยังความเร็วอยุ่เท่าไหร ครับ,enquire,package,0,1
2,ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้ ค่ะ,report,suspend,1,2
3,พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อง...,enquire,internet,0,3
4,ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโท...,report,phone_issues,1,4
...,...,...,...,...,...
16167,ต้องการทราบวันตัดรอบบิลค่ะ,enquire,bill,0,9
16170,เชื่อมต่ออินเตอร์เน็ตไม่ได้ค่ะ,enquire,internet,0,3
16172,ยอดเงินเหลือเท่าไหร่ค่ะ,enquire,balance,0,7
16173,ยอดเงินในระบบ,enquire,balance,0,7


# TODO 2

## USE

In [9]:
import numpy as np
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split

# ฝัง (Embedding) ข้อความด้วย Universal Sentence Encoder (USE)
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
data_df['feature_USE'] = data_df['Sentence_Utterance'].apply(lambda x: use_model([x])[0])

# แปลงข้อมูล feature_USE เป็น numpy array
X_USE = np.array([x.numpy() for x in data_df['feature_USE']])

# แยกข้อมูล Labels สำหรับ Action และ Object
action_labels = data_df['Action_Label'].values
object_labels = data_df['Object_Label'].values

# แบ่งข้อมูล USE เป็น Train/Test/Validation Sets
X_train_use, X_test_use, y_train_action_use, y_test_action_use, y_train_object_use, y_test_object_use = train_test_split(
    X_USE, action_labels, object_labels, test_size=0.2, random_state=42
)
X_train_use, X_val_use, y_train_action_use, y_val_action_use, y_train_object_use, y_val_object_use = train_test_split(
    X_train_use, y_train_action_use, y_train_object_use, test_size=0.2, random_state=42
)

# ตรวจสอบผลลัพธ์
print("USE - Training set shape:", X_train_use.shape)
print("USE - Validation set shape:", X_val_use.shape)
print("USE - Test set shape:", X_test_use.shape)

USE - Training set shape: (8550, 512)
USE - Validation set shape: (2138, 512)
USE - Test set shape: (2672, 512)


## WORD2VEC

In [10]:
import numpy as np
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split

# ฟังก์ชันเพื่อฝังข้อความและจัดการกับคำที่ไม่มีใน Word2Vec model
def get_sentence_embedding(sentence, model):
    word_vectors = [model.wv[word] for word in sentence.split() if word in model.wv]
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)
    else:
        # ถ้าไม่มีคำใน Word2Vec ให้ส่งค่าเวกเตอร์ศูนย์ (300 มิติ)
        return np.zeros(model.vector_size)

# ฝัง (Embedding) ข้อความด้วย Word2Vec โดยใช้ Skip-gram และเพิ่ม epochs
w2v_model = Word2Vec(sentences=[sentence.split() for sentence in data_df['Sentence_Utterance']], 
                     vector_size=300, window=5, min_count=1, workers=4, sg=1, epochs=10)

# ใช้ฟังก์ชัน get_sentence_embedding เพื่อสร้างฟีเจอร์เวกเตอร์สำหรับแต่ละประโยค
data_df['feature_word2vec'] = data_df['Sentence_Utterance'].apply(lambda x: get_sentence_embedding(x, w2v_model))

# แปลงข้อมูล feature_word2vec เป็น numpy array
X_word2vec = np.array(data_df['feature_word2vec'].tolist())

# ตรวจสอบว่ามีค่า NaN ในเวกเตอร์ฝังหรือไม่
if np.isnan(X_word2vec).sum() > 0:
    print("พบค่า NaN ในเวกเตอร์ฝัง")
else:
    print("ไม่มีค่า NaN ในเวกเตอร์ฝัง")

# แยกข้อมูล Labels สำหรับ Action และ Object
action_labels = data_df['Action_Label'].values
object_labels = data_df['Object_Label'].values

# แบ่งข้อมูล Word2Vec เป็น Train/Test/Validation Sets สำหรับ Action และ Object
X_train_w2v, X_test_w2v, y_train_action, y_test_action, y_train_object, y_test_object = train_test_split(
    X_word2vec, action_labels, object_labels, test_size=0.2, random_state=42
)
X_train_w2v, X_val_w2v, y_train_action, y_val_action, y_train_object, y_val_object = train_test_split(
    X_train_w2v, y_train_action, y_train_object, test_size=0.2, random_state=42
)

# ตรวจสอบผลลัพธ์
print("Word2Vec - Training set shape:", X_train_w2v.shape)
print("Word2Vec - Validation set shape:", X_val_w2v.shape)
print("Word2Vec - Test set shape:", X_test_w2v.shape)

ไม่มีค่า NaN ในเวกเตอร์ฝัง
Word2Vec - Training set shape: (8550, 300)
Word2Vec - Validation set shape: (2138, 300)
Word2Vec - Test set shape: (2672, 300)


## วันจันทร์เบิร์ด

In [11]:
import torch
from transformers import AutoTokenizer, AutoModel
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from tqdm import tqdm

# ตรวจสอบว่ามี GPU พร้อมใช้งานหรือไม่
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# โหลด WangchanBERTa และ tokenizer บน GPU
tokenizer = AutoTokenizer.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased")
model = AutoModel.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased").to(device)

# ฟังก์ชันสำหรับการสร้าง embeddings จาก WangchanBERTa แบบ Batch Processing
def get_batch_embeddings(texts, model, tokenizer, batch_size=16):
    model.eval()  # ปิดการเรียนรู้ (no gradients)
    embeddings = []

    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size)):
            batch_texts = texts[i:i+batch_size]
            inputs = tokenizer(batch_texts, return_tensors="pt", padding=True, truncation=True, max_length=128).to(device)
            outputs = model(**inputs)
            batch_embeddings = outputs.last_hidden_state.mean(dim=1).cpu()  # Mean pooling
            embeddings.append(batch_embeddings)

    return torch.cat(embeddings, dim=0).numpy()

# ใช้ WangchanBERTa เพื่อสร้าง embeddings สำหรับข้อมูลทั้งหมด
texts = data_df['Sentence_Utterance'].tolist()
X_wangchanberta = get_batch_embeddings(texts, model, tokenizer, batch_size=16)

# ตรวจสอบว่ามีค่า NaN หรือ Infinite ใน embeddings หรือไม่
if np.isnan(X_wangchanberta).sum() > 0 or np.isinf(X_wangchanberta).sum() > 0:
    print("พบค่า NaN หรือ Infinite ใน embeddings")
else:
    print("ไม่มีค่า NaN หรือ Infinite ใน embeddings")

# ลดมิติของ embeddings ด้วย PCA เพื่อลดขนาดและเพิ่มประสิทธิภาพการฝึกโมเดล
pca = PCA(n_components=128)  # ลดมิติลงเหลือ 128 มิติ
X_wangchanberta_reduced = pca.fit_transform(X_wangchanberta)

# แบ่งข้อมูลเป็น Train, Validation, และ Test sets
y_action = data_df['Action_Label'].values
y_object = data_df['Object_Label'].values

# แบ่งชุดข้อมูลสำหรับ Action และ Object
X_train_wb, X_test_wb, y_train_action, y_test_action, y_train_object, y_test_object = train_test_split(
    X_wangchanberta_reduced, y_action, y_object, test_size=0.2, random_state=42
)

X_train_wb, X_val_wb, y_train_action, y_val_action, y_train_object, y_val_object = train_test_split(
    X_train_wb, y_train_action, y_train_object, test_size=0.2, random_state=42
)

# แสดงข้อมูลเบื้องต้น
print("Training set shape:", X_train_wb.shape)
print("Validation set shape:", X_val_wb.shape)
print("Test set shape:", X_test_wb.shape)

Using device: cuda


tokenizer_config.json:   0%|          | 0.00/282 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/546 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/905k [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/423M [00:00<?, ?B/s]

100%|██████████| 835/835 [00:24<00:00, 34.27it/s]


ไม่มีค่า NaN หรือ Infinite ใน embeddings
Training set shape: (8550, 128)
Validation set shape: (2138, 128)
Test set shape: (2672, 128)


# TODO 3

ก่อนหน้านี้ลองใช้ TF-IDF และ Word2Vec เป็นตัว Embedding และก็ใช้ LR กับ Random Forest ไปแล้วครับแต่ประสิทธิภาพไม่สู้ WangchanBERTa เลยลบทิ้งไป

### วันจันทร์เบิร์ด

In [17]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from sklearn.metrics import classification_report

# สร้างโมเดล Sequential สำหรับ Action Classification
model = Sequential([
    Input(shape=(X_train_wb.shape[1],)),  # ขนาดของ input vector จาก WangchanBERTa
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(8, activation='softmax')  # มี 8 คลาส สำหรับ Action
])

# คอมไพล์โมเดล
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# ฝึกสอนโมเดล
history = model.fit(
    X_train_wb, y_train_action,
    epochs=20,
    batch_size=32,
    validation_data=(X_val_wb, y_val_action),
    verbose=1
)

# ประเมินผลบนชุดทดสอบ
test_loss, test_accuracy = model.evaluate(X_test_wb, y_test_action, verbose=0)
print(f"Test Loss: {test_loss:.2f}, Test Accuracy: {test_accuracy:.2f}")

# ทำนายผลลัพธ์บนชุดทดสอบ
y_pred_action = model.predict(X_test_wb)
y_pred_action_classes = np.argmax(y_pred_action, axis=1)

# แสดงรายงานผลการจำแนกประเภทพร้อมจัดการค่า zero_division
action_labels_names = ['enquire', 'report', 'cancel', 'buy', 'activate', 'request', 'garbage', 'change']
print("\nClassification Report:")
print(classification_report(y_test_action, y_pred_action_classes, target_names=action_labels_names, zero_division=0))

Epoch 1/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 8s 15ms/step - accuracy: 0.4521 - loss: 2.1181 - val_accuracy: 0.6310 - val_loss: 1.2071
Epoch 2/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6335 - loss: 1.2504 - val_accuracy: 0.6389 - val_loss: 1.1183
Epoch 3/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6581 - loss: 1.1306 - val_accuracy: 0.6618 - val_loss: 1.0508
Epoch 4/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6681 - loss: 1.0786 - val_accuracy: 0.6805 - val_loss: 0.9969
Epoch 5/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6897 - loss: 1.0153 - val_accuracy: 0.6904 - val_loss: 0.9754
Epoch 6/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6776 - loss: 1.0099 - val_accuracy: 0.6974 - val_loss: 0.9383
Epoch 7/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.6925 - loss: 0.9493 - val_accuracy: 0.7049 - val_loss: 0.9271
Epoch 8/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.6990 - loss: 0.9232 - val_accuracy: 0

# TODO 4

In [25]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Custom Focal Loss implementation with one-hot encoding for y_true
def focal_loss(gamma=2., alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        epsilon = tf.keras.backend.epsilon()
        y_true = tf.cast(y_true, tf.int32)
        y_true = tf.one_hot(y_true, depth=tf.shape(y_pred)[-1])  # One-hot encoding of y_true
        y_pred = tf.clip_by_value(y_pred, epsilon, 1. - epsilon)
        cross_entropy = -y_true * tf.math.log(y_pred)
        weight = alpha * tf.math.pow(1 - y_pred, gamma)
        loss = weight * cross_entropy
        return tf.reduce_mean(tf.reduce_sum(loss, axis=1))
    return focal_loss_fixed

# Number of classes for Object Classification
num_object_classes = 26

# Calculate class weights to reduce the effect of class imbalance
class_weights = compute_class_weight('balanced', classes=np.unique(y_train_object), y=y_train_object)
class_weight_dict = dict(enumerate(class_weights))

# Create the model for Object classification
inputs = Input(shape=(X_train_wb.shape[1],))

x = Dense(256)(inputs)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(0.3)(x)

x = Dense(128)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(0.2)(x)

x = Dense(64)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(0.2)(x)

x = Dense(32)(x)
x = BatchNormalization()(x)
x = LeakyReLU()(x)
x = Dropout(0.1)(x)

# Output layer for Object Classification
object_output = Dense(num_object_classes, activation='softmax', name='object')(x)

# Build the model
model = Model(inputs=inputs, outputs=object_output)

# Compile the model using Adam optimizer and custom focal loss
optimizer = Adam(learning_rate=0.0005)  # Reduced learning rate for stability
model.compile(optimizer=optimizer,
              loss=focal_loss(gamma=2.0, alpha=0.25),
              metrics=['accuracy'])

# Create callbacks for EarlyStopping and ReduceLROnPlateau
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=1e-6)

# Train the model
history = model.fit(X_train_wb,
                    y_train_object,
                    validation_data=(X_val_wb, y_val_object),
                    epochs=100,
                    batch_size=64,  # Increased batch size for better generalization
                    class_weight=class_weight_dict,
                    callbacks=[early_stopping, reduce_lr])

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test_wb, y_test_object, verbose=2)

print(f"Test Object Accuracy: {test_accuracy:.2f}")

# Make predictions
y_pred_object = model.predict(X_test_wb)

# Convert predictions to class labels
y_pred_object_classes = tf.argmax(y_pred_object, axis=1)

# Classification report for Object
from sklearn.metrics import classification_report
print("\nClassification Report for Object:")
print(classification_report(y_test_object, y_pred_object_classes, zero_division=0))

Epoch 1/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 17s 58ms/step - accuracy: 0.1043 - loss: 0.7628 - val_accuracy: 0.3349 - val_loss: 0.5566 - learning_rate: 5.0000e-04
Epoch 2/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3002 - loss: 0.5559 - val_accuracy: 0.3882 - val_loss: 0.4650 - learning_rate: 5.0000e-04
Epoch 3/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3564 - loss: 0.4503 - val_accuracy: 0.4167 - val_loss: 0.4241 - learning_rate: 5.0000e-04
Epoch 4/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3773 - loss: 0.4369 - val_accuracy: 0.4252 - val_loss: 0.3998 - learning_rate: 5.0000e-04
Epoch 5/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.3974 - loss: 0.4126 - val_accuracy: 0.4275 - val_loss: 0.3856 - learning_rate: 5.0000e-04
Epoch 6/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4016 - loss: 0.3771 - val_accuracy: 0.4350 - val_loss: 0.3737 - learning_rate: 5.0000e-04
Epoch 7/100
134/134 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - a

# TODO 5

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import classification_report

# จำนวนคลาสสำหรับ Action และ Object
num_action_classes = 8  # เปลี่ยนให้ตรงกับจำนวนคลาสของ Action
num_object_classes = 26  # เปลี่ยนให้ตรงกับจำนวนคลาสของ Object

# สร้างโมเดล Multi-task
inputs = Input(shape=(X_train_wb.shape[1],))  # รูปแบบของ embeddings
x = Dense(1000, activation='relu')(inputs)
x = Dropout(0.5)(x)
x = Dense(1000, activation='relu')(x)
x = Dropout(0.5)(x)

# Output layer สำหรับ Action Classification
action_output = Dense(num_action_classes, activation='softmax', name='action')(x)

# Output layer สำหรับ Object Classification
object_output = Dense(num_object_classes, activation='softmax', name='object')(x)

# สร้างโมเดล
model = Model(inputs=inputs, outputs=[action_output, object_output])
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss={
        'action': 'sparse_categorical_crossentropy',
        'object': 'sparse_categorical_crossentropy'
    },
    metrics={
        'action': 'accuracy',  # สำหรับการจำแนกประเภท Action
        'object': 'accuracy'   # สำหรับการจำแนกประเภท Object
    }
)

# แสดงสรุปของโมเดล
model.summary()

# ฝึกโมเดล
history = model.fit(
    X_train_wb, 
    {'action': y_train_action, 'object': y_train_object}, 
    validation_data=(X_val_wb, {'action': y_val_action, 'object': y_val_object}),
    epochs=20, 
    batch_size=32
)

# ประเมินผลบนชุดทดสอบ
test_results = model.evaluate(
    X_test_wb, 
    {'action': y_test_action, 'object': y_test_object}, 
    verbose=2
)

# แสดงผลลัพธ์ทั้งหมดที่คืนมา
print("Test Results:", test_results)

# ตรวจสอบจำนวนค่าที่คืนมา
num_results = len(test_results)
print(f"Number of results returned: {num_results}")

# ถ้าจำนวนค่าที่คืนมามากกว่าหรือน้อยกว่าที่คาดไว้ ให้ใช้การตรวจสอบเพื่อหลีกเลี่ยง IndexError
if num_results == 5:
    test_loss, test_action_loss, test_object_loss, test_action_accuracy, test_object_accuracy = test_results
elif num_results == 3:
    test_loss, test_action_loss, test_object_loss = test_results
    test_action_accuracy = None  # หรือสามารถกำหนดค่า default เช่น 0.0
    test_object_accuracy = None
else:
    print("Unexpected number of results returned from evaluation.")

# แสดงผลการประเมิน
if test_action_accuracy is not None and test_object_accuracy is not None:
    print(f"Test Action Loss: {test_action_loss:.2f}, Test Action Accuracy: {test_action_accuracy:.2f}")
    print(f"Test Object Loss: {test_object_loss:.2f}, Test Object Accuracy: {test_object_accuracy:.2f}")

# ทำการทำนายผลลัพธ์
y_pred_action, y_pred_object = model.predict(X_test_wb)

# แปลงการทำนายผลลัพธ์ให้เป็นคลาส
y_pred_action_classes = tf.argmax(y_pred_action, axis=1).numpy()
y_pred_object_classes = tf.argmax(y_pred_object, axis=1).numpy()

# Mapping dictionaries ที่ใช้ในการแปลงค่ากลับเป็นชื่อเดิม
action_mapping = {0: 'enquire', 1: 'report', 2: 'cancel', 3: 'buy', 4: 'activate', 5: 'request', 6: 'garbage', 7: 'change'}
object_mapping = {0: 'payment', 1: 'package', 2: 'suspend', 3: 'internet', 4: 'phone_issues', 5: 'service', 
                  6: 'nontruemove', 7: 'balance', 8: 'detail', 9: 'bill', 10: 'credit', 11: 'promotion', 
                  12: 'mobile_setting', 13: 'iservice', 14: 'roaming', 15: 'truemoney', 
                  16: 'information', 17: 'lost_stolen', 18: 'balance_minutes', 19: 'idd', 
                  20: 'garbage', 21: 'ringtone', 22: 'rate', 23: 'loyalty_card', 24: 'contact', 25: 'officer'}

# แปลงค่ากลับเป็นชื่อเดิม
y_pred_action_names = [action_mapping[class_id] for class_id in y_pred_action_classes]
y_pred_object_names = [object_mapping[class_id] for class_id in y_pred_object_classes]

# รายงานการจำแนกประเภทสำหรับ Action
print("\nClassification Report for Action:")
print(classification_report(y_test_action, y_pred_action_classes, target_names=list(action_mapping.values())))

# รายงานการจำแนกประเภทสำหรับ Object
print("\nClassification Report for Object:")
print(classification_report(y_test_object, y_pred_object_classes, target_names=list(object_mapping.values())))

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 128)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1000)      │    129,000 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_5 (Dropout) │ (None, 1000)      │          0 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 1000)      │  1,001,000 │ dropout_5[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_6 (Dropout) │ (None, 1000)      │          0 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ action (Dense)      │ (None, 8)         │      8,008 │ dropout_6[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ object (Dense)      │ (None, 26)        │     26,026 │ dropout_6[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 1,164,034 (4.44 MB)

 Trainable params: 1,164,034 (4.44 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 11s 19ms/step - action_accuracy: 0.6186 - loss: 3.8846 - object_accuracy: 0.2641 - val_action_accuracy: 0.6978 - val_loss: 2.8307 - val_object_accuracy: 0.4397
Epoch 2/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - action_accuracy: 0.7147 - loss: 2.7714 - object_accuracy: 0.4280 - val_action_accuracy: 0.7198 - val_loss: 2.6325 - val_object_accuracy: 0.4771
Epoch 3/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - action_accuracy: 0.7440 - loss: 2.4862 - object_accuracy: 0.4786 - val_action_accuracy: 0.7175 - val_loss: 2.5749 - val_object_accuracy: 0.4771
Epoch 4/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - action_accuracy: 0.7650 - loss: 2.3609 - object_accuracy: 0.4927 - val_action_accuracy: 0.7259 - val_loss: 2.5671 - val_object_accuracy: 0.4855
Epoch 5/20
268/268 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - action_accuracy: 0.7691 - loss: 2.2234 - object_accuracy: 0.5222 - val_action_accuracy: 0.7306 - val_loss: 2.5320 - val_object_accuracy: 0.4935
Epoch 6/

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classif